In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 107.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [2]:
import transformers
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

print(transformers.__version__)
# 4.42.3 -> 4.43.2
# transformers >= 4.43.0 required

4.46.3


In [3]:
import json
from pprint import pprint
# Open and read the JSON file
with open('/kaggle/input/viquadv1/ViQuAD/dev_ViQuAD.json', 'r') as file:
    data_dev = json.load(file)
    
with open('/kaggle/input/viquadv1/ViQuAD/test_ViQuAD.json', 'r') as file:
    data_test = json.load(file)
    
# keys = data['data'][0].keys()
# print(keys)

# Print the data
# pprint(data["data"][1])

In [4]:
from collections import Counter

def normalize_text(s):
    import string, re

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))

def exact_match(prediction, ground_truth):
    return int(normalize_text(prediction) == normalize_text(ground_truth))

def F1_score(predict, ground_truth):
  predict_tokens = normalize_text(predict).split()
  ground_truth_tokens = normalize_text(ground_truth).split()
  common = Counter(predict_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(predict_tokens)
  # print(precision)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  # print(recall)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [5]:
#Zero-shot prompt
def zero_shot_prompt(context, question):
    prompt = f"""
    Read the following text and answer the question below. The answer must be within the paragraph and be one continuous phrase or keyword, not separated, without explanation or additional information outside the paragraph.   
    Văn bản:
    {context}
    
    Câu hỏi:
    {question}
    
    Trả lời:
    """
    
    
    return(prompt)

#One-shot prompt
def one_shot_prompt(context, question):
    prompt = f"""
    Read the following text and answer the question below. The answer must be within the paragraph and be one continuous phrase or keyword, not separated, without explanation or additional information outside the paragraph.   
    
    Ví dụ:
    Văn bản: "Albert Einstein là nhà vật lý nổi tiếng với thuyết tương đối."
    Câu hỏi: "Ai là người phát triển thuyết tương đối?"
    Trả lời: "Albert Einstein"
    
    Văn bản:
    {context}
    
    Câu hỏi:
    {question}
    
    Trả lời:
    """
    
    
    return(prompt)
#Few_shot prompt
def few_shot_prompt(context, question):
    prompt = f"""
    Read the following text and answer the question below. The answer must be within the paragraph and be one continuous phrase or keyword, not separated, without explanation or additional information outside the paragraph.   
    
    Ví dụ:
    Văn bản: "Albert Einstein là nhà vật lý nổi tiếng với thuyết tương đối."
    Câu hỏi: "Ai là người phát triển thuyết tương đối?"
    Trả lời: "Albert Einstein"
    
    Văn bản: "Kháng sinh được dùng để điều trị các bệnh nhiễm trùng do vi khuẩn gây ra."
    Câu hỏi: "Kháng sinh dùng để điều trị cái gì?"
    Trả lời: "nhiễm trùng do vi khuẩn"
    
    Văn bản: "Isaac Newton đã phát hiện ra định luật vạn vật hấp dẫn vào thế kỷ 17."
    Câu hỏi: "Định luật vạn vật hấp dẫn được phát hiện vào thời kỳ nào?"
    Trả lời: "thế kỷ 17"
    
    Văn bản: "Paris là thủ đô của Pháp và là một trung tâm văn hóa lớn."
    Câu hỏi: "Paris là thủ đô của quốc gia nào?"
    Trả lời: "Pháp"
    
    Văn bản: "Con người khám phá ra lửa có thể được tạo ra bằng cách đánh đá vào nhau vì nó tạo ra tia lửa."
    Câu hỏi: "Tại sao lửa có thể được tạo ra khi đánh đá vào nhau?"
    Trả lời: "tạo ra tia lửa"
    
    Văn bản: "Việc xây dựng cây cầu mất 5 năm do địa hình phức tạp và thời tiết xấu."
    Câu hỏi: "Việc xây dựng cây cầu mất bao lâu?"
    Trả lời: "5 năm"
    
    Văn bản:
    {context}
    
    Câu hỏi:
    {question}
    
    Trả lời:
    """
    
    
    return(prompt)

#Chaiprompt prompt
def chain_of_thought_prompt(context, question):
    prompt = f"""
    Read the following context and answer the question below. Follow these steps to ensure accuracy:
    
    1. Read and fully understand the provided context.
    2. Identify the sentence or sentences in the context that may contain the answer to the question.
    3. Analyze the sentence(s) to confirm the exact information that answers the question.
    4. Provide the exact phrase or continuous keyword from the sentence(s) that answers the question, ensuring the answer is unbroken and directly addresses the question.
    5. Ensure no additional explanation or words are included outside of the exact phrase.
    Văn bản:
    {context}
    
    Câu hỏi:
    {question}
    
    Trả lời:
    """
    
    
    return(prompt)



In [6]:
model_id = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from tqdm import tqdm
import json

def evaluate_dev(technique_prompt, data):
    data_eva = data.copy()
    file_name = technique_prompt.__name__ + "_dev_evaluate.json"
    print(file_name)
    for _, data in tqdm(enumerate(data_eva["data"]), total=len(data_eva["data"])):
        for _, paragraph in tqdm(enumerate(data["paragraphs"]), total=len(data["paragraphs"])):
          context = paragraph["context"]
          for _, qa in (enumerate(paragraph["qas"])):
            question = qa["question"]
            prompt = technique_prompt(context, question)
            messages = [
                # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
                {"role": "user", 
                 "content": prompt
                },
            ]
            
            outputs = pipeline(
                messages,
                max_new_tokens=256,
            )
            
            response = outputs[0]["generated_text"][-1]['content']
            em_score = max((exact_match(response, answer['text'])) for _, answer in enumerate(qa["answers"]))
            f1_score = max((F1_score(response, answer['text'])) for _, answer in enumerate(qa["answers"]))
            qa['answer_predict'] = response
            qa['em_score'] = em_score
            qa['f1_score'] =f1_score
            # print(response)
            # print(em_score, f1_score)
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data_eva, f, ensure_ascii=False, indent=4)
        
def evaluate_test(technique_prompt, data):
    data_eva = data.copy()
    file_name = technique_prompt.__name__ + "_test_evaluate.json"
    print(file_name)
    for _, data in tqdm(enumerate(data_eva["data"]), total=len(data_eva["data"])):
        for _, paragraph in tqdm(enumerate(data["paragraphs"]), total=len(data["paragraphs"])):
          context = paragraph["context"]
          for _, qa in (enumerate(paragraph["qas"])):
            question = qa["question"]
            prompt = technique_prompt(context, question)
            messages = [
                # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
                {"role": "user", 
                 "content": prompt
                },
            ]
            
            outputs = pipeline(
                messages,
                max_new_tokens=256,
            )
            
            response = outputs[0]["generated_text"][-1]['content']
            em_score = max((exact_match(response, answer['text'])) for _, answer in enumerate(qa["answers"]))
            f1_score = max((F1_score(response, answer['text'])) for _, answer in enumerate(qa["answers"]))
            qa['answer_predict'] = response
            qa['em_score'] = em_score
            qa['f1_score'] =f1_score
            # print(response)
            # print(em_score, f1_score)

    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data_eva, f, ensure_ascii=False, indent=4)

In [8]:
#evaluate_dev(zero_shot_prompt, data_dev)
#evaluate_dev(one_shot_prompt, data_dev)
#evaluate_dev(few_shot_prompt, data_dev)
#evaluate_dev(chain_of_thought_prompt, data_dev)

In [9]:
evaluate_test(zero_shot_prompt, data_test)
evaluate_test(one_shot_prompt, data_test)
evaluate_test(few_shot_prompt, data_test)
evaluate_test(chain_of_thought_prompt, data_test)

zero_shot_prompt_test_evaluate.json


  0%|          | 0/78 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  1%|▏         | 1/78 [00:04<05:19,  4.15s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  3%|▎         | 2/78 [00:07<04:17,  3.39s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for

one_shot_prompt_test_evaluate.json


  0%|          | 0/78 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  1%|▏         | 1/78 [00:03<04:02,  3.14s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  3%|▎         | 2/78 [00:05<03:45,  2.96s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_tok

few_shot_prompt_test_evaluate.json


  0%|          | 0/78 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  1%|▏         | 1/78 [00:03<04:38,  3.62s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  3%|▎         | 2/78 [00:08<05:15,  4.15s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_tok

chain_of_thought_prompt_test_evaluate.json


  0%|          | 0/78 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  1%|▏         | 1/78 [00:03<04:07,  3.21s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

  3%|▎         | 2/78 [00:07<05:09,  4.08s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_tok

In [10]:
# for _, data in enumerate(data_dev['data'][0]['paragraphs'][0]['qas']):
#     prompt = f"""
#         Read the following text and answer the question below. The answer must be within the paragraph and be one continuous phrase or keyword, not separated, without explanation or additional information outside the paragraph.   
#         Văn bản:
#         {data_dev['data'][0]['paragraphs'][0]['context']}
        
#         Câu hỏi:
#         {data['question']}
        
#         Trả lời:
#       """

#     messages = [
#         # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#         {"role": "user", 
#          "content": prompt
#         },
#     ]
    
#     outputs = pipeline(
#         messages,
#         max_new_tokens=256,
#     )
    
#     response = outputs[0]["generated_text"][-1]['content']
#     em_score = max((exact_match(response, answer['text'])) for _, answer in enumerate(data["answers"]))
#     f1_score = max((F1_score(response, answer['text'])) for _, answer in enumerate(data["answers"]))
#     print(em_score, f1_score)
#     print(response)